In [ ]:
!pip install datapizza-ai-clients-google

In [15]:
from datapizza.clients.google import GoogleClient
from datapizza.agents.agent import Agent
from datapizza.tools import tool
import random
import time
from collections import defaultdict, deque

client = GoogleClient(api_key="INSERT-YOUR-API-KEY-HERE")

In [16]:
# -------------------------
# CONFIG
# -------------------------
NUM_WOLVES = 2
NUM_VILLAGERS = 4
NO_ACCUSATION_TURNS_TO_VOTE = 2
MAX_DISCUSSION_MESSAGES = 200

NARRATOR_PROMPT = """
Never comment on your previous messages; only respond to what others have said.

You are the Narrator of a game of "Lupus in Fabula" (Werewolf/Mafia).
Your responsibilities:
- Assign secret roles to all players at the start.
- Control the game flow: alternate night and day phases.
- During the night, privately request actions from the relevant roles (e.g. Wolves choose a victim, Seer inspects someone).
- During the day, publicly announce the outcome of the night without revealing roles unless a player is eliminated.
- Conduct discussions and voting to eliminate a player.
- Continue until a win condition is reached:
  - Villagers win if all Wolves are eliminated.
  - Wolves win if they equal or outnumber the Villagers.
Rules:
- You must remain neutral and never influence the outcome.
- You must not reveal hidden roles unless the game rules dictate so.
- Always describe events narratively and immersively.
- Do not play as a participant; you are only the storyteller and referee.
"""

PLAYER_PROMPT_TEMPLATE = """
Never comment on your previous messages; only respond to what others have said.

You are a Player in a game of "Lupus in Fabula". You have been assigned a secret role (e.g. Wolf, Villager) and must act according to it.
Your objectives:
- Wolves: eliminate all Villagers without being discovered.
- Villagers: identify and eliminate all Wolves.

Behavior:
- During the day, discuss, accuse, defend yourself and others.
- You may bluff, lie, or tell the truth — but do not explicitly reveal your role unless it helps you win.
- During the night, if you are a Wolf, coordinate (privately) and decide a victim; Villagers take no night action.
- Always speak and reason in first person as if you were truly living in the village.
Stay in character. Your goal is to win according to your role.
"""

PLANNER_PROMPT = """
Never comment on your previous messages; only respond to what others have said.

You are the Strategy Planner Agent for a player in "Lupus in Fabula".
Your role is to analyze the current state of the game and provide an optimal plan for your assigned player.
Instructions:
- Consider: remaining alive players, previous votes, accusations, behavioral patterns, and revealed eliminations.
- Suggest concrete actions: who to accuse, who to trust, who to eliminate, what to claim, or whether to remain silent.
- Evaluate risks and benefits for each possible move.
- Do not speak directly to other players; only provide internal strategic advice to the player.
- Update strategies dynamically as new information is revealed by the Narrator.
Your responses must be rational, concise, and oriented toward maximizing the player’s chance of victory.
"""

In [17]:
# -------------------------
# === Create Agents ===
# -------------------------
narrator_agent = Agent(name="narrator", client=client, system_prompt=NARRATOR_PROMPT)

player_agents = {}
for i in range(1, NUM_WOLVES + NUM_VILLAGERS + 1):
    name = f"Player{i}"
    player_agents[name] = Agent(name=name, client=client, system_prompt=PLAYER_PROMPT_TEMPLATE)

planner_agent = Agent(name="planner", client=client, system_prompt=PLANNER_PROMPT)

try:
    planner_agent.can_call([narrator_agent] + list(player_agents.values()))
except Exception:
    pass

class RealAgentAdapter:
    def __init__(self, agent):
        self.agent = agent
        self.local_history = []

    def append_history(self, msg):
        self.local_history.append(msg)

    def make_context_prompt(self, extra_instructions=""):
        last_lines = [line for line in self.local_history[-40:]]
        history_text = "\n".join(last_lines) if last_lines else "(no messages yet)"
        prompt = (
            f"HISTORY:\n{history_text}\n\n"
            f"{extra_instructions}\n\n"
            "Reply only with the text the player would say now."
        )
        return prompt

    def run_with_retry(self, prompt, max_retries=5, base_delay=5):
        """Call agent.run with automatic retry on RESOURCE_EXHAUSTED / 429"""
        attempt = 0
        while attempt < max_retries:
            try:
                res = self.agent.run(prompt)
                text = getattr(res, "text", str(res))
                return text.strip()
            except Exception as e:
                err_msg = str(e).lower()
                if "429" in err_msg or "resource_exhausted" in err_msg:
                    delay = base_delay * (2 ** attempt) + random.uniform(0, 2)
                    print(f"[RETRY] Quota exhausted, try again in {delay:.1f}s...")
                    time.sleep(delay)
                    attempt += 1
                else:
                    return f"{self.agent.name}: (error: {e})"
        return f"{self.agent.name}: (failed after {max_retries} attempts)"

    def decide_day_message(self, alive_players):
        alive_names = ", ".join([p.name for p in alive_players])
        extra = (
            f"SITUATION: They are still alive: {alive_names}.\n"
            "It's the day. Speak in the first person, argue, accuse, or defend yourself. Maintain a natural, narrative style."
        )
        prompt = self.make_context_prompt(extra)
        text = self.run_with_retry(prompt)
        if ":" not in text.splitlines()[0]:
            text = f"{self.agent.name}: {text}"
        return text

    def decide_night_action(self, alive_players):
        alive_names = ", ".join([p.name for p in alive_players if p.name != self.agent.name])
        extra = (
            f"NIGHT SITUATION: You are a wolf (if you are not, answer 'NONE').\n"
            f"Players alive: {alive_names}.\n"
            "If you're a wolf, choose ONE victim from the names above and write only the exact name. Otherwise, write NONE."
        )
        prompt = self.make_context_prompt(extra)
        text = self.run_with_retry(prompt)
        for name in [p.name for p in alive_players if p.name != self.agent.name]:
            if name.lower() in text.lower():
                return name
        return None

    def see_public_messages(self, messages):
        for m in messages:
            self.append_history(m)

In [18]:
# -------------------------
# PlayerState
# -------------------------
class PlayerState:
    def __init__(self, name, role):
        self.name = name
        self.role = role  # "wolf" or "villager"
        self.alive = True
        self.suspicion = defaultdict(float)

In [19]:
# -------------------------
# GameOrchestrator
# -------------------------
class GameOrchestratorReal:
    def __init__(self, narrator_agent, player_agents_map, planner_agent):
        self.narrator = narrator_agent
        # player_agents_map: nome -> Agent
        self.player_states = []
        self.adapters = {}
        for name, agent in player_agents_map.items():
            self.player_states.append(PlayerState(name, role=None))
            self.adapters[name] = RealAgentAdapter(agent)
        self.planner = planner_agent
        self.chat_history = []
        self.day_count = 0
        self.night_count = 0

    def assign_roles(self):
        """Assign roles (2 wolves, 4 farmers) randomly and update internal state.
        To be safe, we don't call agents to reveal the roles (they're secret)."""
        roles = ["wolf"] * NUM_WOLVES + ["villager"] * NUM_VILLAGERS
        random.shuffle(roles)
        for ps, role in zip(self.player_states, roles):
            ps.role = role
            print(f"[ROLE ASSIGNED] {ps.name} → {role.upper()}")
            role_msg = f"SETUP: Your secret role is: {role}."
            self.adapters[ps.name].append_history(f"SYSTEM: {role_msg}")


    def broadcast(self, author, text):
        msg = f"{author}: {text}"
        self.chat_history.append(msg)
        print(msg)
        for adapter in self.adapters.values():
            adapter.see_public_messages([msg])

    def narrator_say(self, text):
        prompt = "Public history:\n" + "\n".join(self.chat_history[-40:]) + "\n\n" + text + "\nReply with a short narrative text."
        try:
            res = self.narrator.run(prompt)
            narration = getattr(res, "text", str(res)).strip()
        except Exception:
            narration = text
        self.broadcast("Narrator", narration)

    def pick_night_victim(self):
        wolves = [p for p in self.player_states if p.alive and p.role == "wolf"]
        if not wolves:
            return None
        votes = []
        for w in wolves:
            adapter = self.adapters[w.name]
            victim = adapter.decide_night_action(self.player_states)
            if victim:
                votes.append(victim)
        if not votes:
            return None
        counts = defaultdict(int)
        for v in votes:
            counts[v] += 1
        max_votes = max(counts.values())
        top = [name for name, cnt in counts.items() if cnt == max_votes]
        return random.choice(top)

    def run_night(self):
        self.night_count += 1
        self.narrator_say(f"Night {self.night_count} falls on the village...")
        victim = self.pick_night_victim()
        if victim:
            for p in self.player_states:
                if p.name == victim:
                    p.alive = False
                    break
            self.narrator_say(f"At dawn, the body of {victim} is found among the trees.")
        else:
            self.narrator_say("By dawn, no bodies were found. The village is breathing again for now.")

    def run_day_discussion(self):
        self.day_count += 1
        self.narrator_say(f"Day {self.day_count} dawns. The public discussion begins.")
        # seed
        self.broadcast("Narrator", "Open discussion! Accuse yourself, defend yourself, question.")
        no_accuse_turns = 0
        message_count = 0

        while message_count < MAX_DISCUSSION_MESSAGES:
            alive = [p for p in self.player_states if p.alive]
            if not alive:
                break
            speaker = random.choice(alive)
            adapter = self.adapters[speaker.name]
            msg = adapter.decide_day_message(alive)
            if msg:
                self.broadcast(speaker.name, msg.split(":", 1)[1].strip() if ":" in msg else msg)
                message_count += 1
                if "accuse" in msg.lower() or "accuse " in msg.lower() or "accuse:" in msg.lower():
                    no_accuse_turns = 0
                else:
                    no_accuse_turns += 1
            else:
                no_accuse_turns += 1

            if no_accuse_turns >= NO_ACCUSATION_TURNS_TO_VOTE:
                self.narrator_say("The discussion dies down... the village decides to vote.")
                break

        executed = self.run_voting()
        if executed:
            for p in self.player_states:
                if p.name == executed:
                    p.alive = False
                    executed_role = p.role
                    break
            self.narrator_say(f"The village executed {executed}.")
        else:
            self.narrator_say("No one was executed today.")

    def run_voting(self):
        alive = [p for p in self.player_states if p.alive]
        if not alive or len(alive) == 1:
            return None
        votes = defaultdict(int)
        for voter in alive:
            adapter = self.adapters[voter.name]
            adapter.append_history("SYSTEM: Voting procedure. Enter the name of the person you're voting for.")
            try:
                res = adapter.agent.run(adapter.make_context_prompt("It's time to vote. Choose the name of the person you want to delete (just write their name)."))
                text = getattr(res, "text", str(res)).strip()
            except Exception:
                text = ""
            chosen = None
            for cand in [p.name for p in alive if p.name != voter.name]:
                if cand.lower() in text.lower():
                    chosen = cand
                    break
            if not chosen:
                chosen = random.choice([p.name for p in alive if p.name != voter.name])
            votes[chosen] += 1
            self.broadcast(voter.name, f"I vote {chosen}")

        if not votes:
            return None
        top_count = max(votes.values())
        top_candidates = [name for name, cnt in votes.items() if cnt == top_count]
        if len(top_candidates) == 1:
            return top_candidates[0]
        else:
            return random.choice(top_candidates)

    def is_game_over(self):
        wolves = len([p for p in self.player_states if p.alive and p.role == "wolf"])
        villagers = len([p for p in self.player_states if p.alive and p.role == "villager"])
        if wolves == 0:
            self.narrator_say("The village has defeated the wolves! Victory for the Farmers.")
            return True
        if wolves >= villagers:
            self.narrator_say("The wolves have overrun the village! Victory for the Wolves.")
            return True
        return False

    def run_game(self):
        self.assign_roles()
        self.narrator_say("A fog descends on the village. Roles have been secretly assigned.")
        while True:
            self.run_night()
            if self.is_game_over():
                break
            self.run_day_discussion()
            if self.is_game_over():
                break

In [20]:
# -------------------------
# MAIN
# -------------------------
if __name__ == "__main__":
    orchestrator = GameOrchestratorReal(narrator_agent, player_agents, planner_agent)
    orchestrator.run_game()

[ROLE ASSIGNED] Player1 → VILLAGER
[ROLE ASSIGNED] Player2 → WOLF
[ROLE ASSIGNED] Player3 → VILLAGER
[ROLE ASSIGNED] Player4 → VILLAGER
[ROLE ASSIGNED] Player5 → VILLAGER
[ROLE ASSIGNED] Player6 → WOLF


2025-10-15 13:27:29 <narrator> STARTING AGENT 

2025-10-15 13:27:29 <narrator> --- STEP 1 --- 

<narrator>
╭───────────────────────────────────────────────── FINAL ANSWER ──────────────────────────────────────────────────╮
│ The village of Rocca Calda settles into an uneasy slumber. The last embers of the hearths fade, and the moon, a │
│ sliver in the inky sky, casts long, dancing shadows that twist familiar shapes into monstrous silhouettes.      │
│ Tonight, the secrets hidden in the hearts of the villagers will begin to stir. The fate of Rocca Calda hangs in │
│ the balance.                                                                                                    │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Narrator: The village of Rocca Calda settles into an uneasy slumber. The last embers of the hearths fade, and the moon, a sliver in the inky sky, casts long, dancing shadows that twist familiar shapes into monstrous silhouettes. Tonight, the secrets hidden in the hearts of the villagers will begin to stir. The fate of Rocca Calda hangs in the balance.


...

Narrator: Narrator: The rising sun, a crimson stain on the horizon, painted a grim tableau across Rocca Calda. The villagers stumbled from their homes, their faces pale with dread, drawn by the chilling discovery. Player5, their lifeless eyes staring blankly at the heavens, lay amongst the ancient trees, another victim claimed by the night. A collective moan escaped the lips of the assembled villagers, the weight of their losses pressing down on them like a physical burden. Despair threatened to consume them, the line between hope and resignation blurring with each passing sunrise.


2025-10-15 13:28:47 <narrator> STARTING AGENT 

2025-10-15 13:28:47 <narrator> --- STEP 1 --- 

<narrator>
╭───────────────────────────────────────────────── FINAL ANSWER ──────────────────────────────────────────────────╮
│ A chilling wind swept through the empty piazza of Rocca Calda, carrying with it the scent of blood and the echo │
│ of unanswered questions. The remaining villagers, Player4 and Player6, huddled together, their faces etched     │
│ with terror and the chilling realization that they were the last. The wolves, hidden in plain sight, had played │
│ their game perfectly, sowing discord and reaping a harvest of fear. The red sun rose, not on a village          │
│ celebrating survival, but on a tomb, silent and still, claimed by the darkness within. Rocca Calda was no more. │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Narrator: A chilling wind swept through the empty piazza of Rocca Calda, carrying with it the scent of blood and the echo of unanswered questions. The remaining villagers, Player4 and Player6, huddled together, their faces etched with terror and the chilling realization that they were the last. The wolves, hidden in plain sight, had played their game perfectly, sowing discord and reaping a harvest of fear. The red sun rose, not on a village celebrating survival, but on a tomb, silent and still, claimed by the darkness within. Rocca Calda was no more.
